In [91]:
import pandas as pd
import numpy as np

In [92]:
wildfire_df=pd.read_csv('../Datasets/wildfire_df.csv')
wildfire_columns=wildfire_df.columns
wildfire_df=wildfire_df[(wildfire_df['state']=='California') & (wildfire_df['acq_date']>'2020-01-01') & 
                        (wildfire_df['latitude']<=36)].copy()
# Dropping rows where same day and same time there were multiple rows
wildfire_df.dropna(subset=['acq_time','acq_date'],inplace=True)

# fixing and typecasting date and time column  properly
wildfire_df['acq_date']=pd.to_datetime(wildfire_df['acq_date']).dt.date
wildfire_df['acq_time'] = wildfire_df['acq_time'].astype(str).str.zfill(4)  # Ensure leading zeros for hours < 10
wildfire_df['acq_time'] = pd.to_datetime(wildfire_df['acq_time'], format='%H%M').dt.time
wildfire_df['acq_time']= pd.to_datetime(wildfire_df['acq_time'], format='%H:%M:%S')
wildfire_df['nearest_hour']=wildfire_df['acq_time'].dt.round('H').dt.time
wildfire_df['acq_time']=wildfire_df['acq_time'].dt.time
wildfire_df

C:\Users\Olan\AppData\Local\Temp\ipykernel_18504\1733741748.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  wildfire_df=pd.read_csv('../Datasets/wildfire_df.csv')


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,state,nearest_hour
5276400,33.77762,-118.22819,297.45,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,283.34,1.14,N,0.0,California,09:00:00
5276401,33.77818,-118.23756,298.90,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,285.94,1.01,N,0.0,California,09:00:00
5276402,33.81285,-118.23906,297.82,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,285.95,0.99,N,0.0,California,09:00:00
5276403,33.81887,-118.24325,296.60,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,285.98,1.08,N,2.0,California,09:00:00
5276404,33.81951,-118.24489,298.51,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,2,286.40,0.92,N,2.0,California,09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8054566,34.32040,-118.51376,301.29,0.39,0.44,2024-09-01,09:20:00,N21,VIIRS,n,2.0NRT,290.06,0.39,N,0.0,California,09:00:00
8054567,34.33338,-118.52016,311.39,0.39,0.44,2024-09-01,09:20:00,N21,VIIRS,n,2.0NRT,290.80,1.18,N,0.0,California,09:00:00
8054568,34.42942,-118.64478,311.10,0.40,0.44,2024-09-01,09:20:00,N21,VIIRS,n,2.0NRT,290.99,1.34,N,0.0,California,09:00:00
8054569,34.60736,-117.33656,317.76,0.52,0.42,2024-09-01,09:20:00,N21,VIIRS,n,2.0NRT,295.19,3.13,N,0.0,California,09:00:00


In [93]:
hourly_weather=pd.read_csv('../Datasets/hourly_weather_data_complete.csv',index_col=False)
hourly_weather['acq_date']=pd.to_datetime(hourly_weather['date']).dt.date
hourly_weather['time']=pd.to_datetime(hourly_weather['date']).dt.time
hourly_weather

,date,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,...,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,acq_date,time
0,2020-01-02 07:00:00+00:00,33.77762,-118.22819,10.910,95.444640,10.210,9.915919,0.0,0.0,0.0,...,11.360001,13.410000,13.960000,21.910,0.274,0.319,0.195,0.026,2020-01-02,07:00:00
1,2020-01-02 08:00:00+00:00,33.77762,-118.22819,11.010,96.407850,10.460,9.846991,0.0,0.0,0.0,...,11.060000,13.260000,14.010000,21.910,0.275,0.319,0.195,0.026,2020-01-02,08:00:00
2,2020-01-02 09:00:00+00:00,33.77762,-118.22819,10.460,95.428820,9.760,9.326071,0.0,0.0,0.0,...,10.760000,13.110001,14.010000,21.910,0.275,0.318,0.195,0.026,2020-01-02,09:00:00
3,2020-01-02 10:00:00+00:00,33.77762,-118.22819,10.160,95.739235,9.510,9.024604,0.0,0.0,0.0,...,10.360001,12.960000,14.010000,21.910,0.276,0.318,0.195,0.026,2020-01-02,10:00:00
4,2020-01-02 11:00:00+00:00,33.77762,-118.22819,9.010,97.006090,8.560,7.496633,0.0,0.0,0.0,...,9.960000,12.810000,14.010000,21.910,0.276,0.318,0.195,0.026,2020-01-02,11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449955,2024-09-02 02:00:00+00:00,35.03100,-118.31759,31.249,16.880789,3.199,27.736122,0.0,0.0,0.0,...,38.249000,31.598999,28.549000,25.549,0.022,0.149,0.213,0.187,2024-09-02,02:00:00
1449956,2024-09-02 03:00:00+00:00,35.03100,-118.31759,28.699,18.999668,2.799,24.775993,0.0,0.0,0.0,...,35.499000,31.699000,28.549000,25.549,0.022,0.149,0.213,0.187,2024-09-02,03:00:00
1449957,2024-09-02 04:00:00+00:00,35.03100,-118.31759,27.449,20.800364,3.049,23.649523,0.0,0.0,0.0,...,33.149000,31.699000,28.549000,25.549,0.022,0.149,0.213,0.187,2024-09-02,04:00:00
1449958,2024-09-02 05:00:00+00:00,35.03100,-118.31759,26.949,21.495300,3.099,23.265472,0.0,0.0,0.0,...,31.199000,31.699000,28.598999,25.549,0.022,0.149,0.213,0.187,2024-09-02,05:00:00


In [98]:
# Dropping instances where time is NaN - Data was not available in the meteostat api
# Merging wildfire and weather data gives us all occurences where wildfires occured
positive_samples=wildfire_df.merge(hourly_weather,left_on=['latitude','longitude','acq_date','nearest_hour'],right_on=['latitude','longitude','acq_date','time'],how='left').drop_duplicates().dropna(subset=['time'])

In [99]:
# dropping where confidence ='l'
positive_samples=positive_samples[positive_samples['confidence']!='l'].copy()

In [100]:
# setting target label = 1 
positive_samples['fire']=1

In [101]:
positive_samples

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,...,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,time,fire
0,33.77762,-118.22819,297.45,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,...,10.760000,13.110001,14.010000,21.910000,0.275,0.318,0.195,0.026,09:00:00,1
1,33.77818,-118.23756,298.90,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,...,10.753500,13.103500,14.003500,21.903500,0.275,0.318,0.195,0.026,09:00:00,1
2,33.81285,-118.23906,297.82,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,...,10.714500,13.064501,13.964500,21.864500,0.275,0.318,0.195,0.026,09:00:00,1
3,33.81887,-118.24325,296.60,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,...,10.423500,12.823500,13.723500,21.923500,0.273,0.324,0.201,0.020,09:00:00,1
4,33.81951,-118.24489,298.51,0.42,0.61,2020-01-02,08:53:00,N,VIIRS,n,...,10.443000,12.842999,13.743000,21.942999,0.273,0.324,0.201,0.020,09:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62046,34.32040,-118.51376,301.29,0.39,0.44,2024-09-01,09:20:00,N21,VIIRS,n,...,24.445000,29.695000,27.244999,24.445000,0.075,0.145,0.165,0.188,09:00:00,1
62047,34.33338,-118.52016,311.39,0.39,0.44,2024-09-01,09:20:00,N21,VIIRS,n,...,24.536000,29.786000,27.335999,24.536000,0.075,0.145,0.165,0.188,09:00:00,1
62048,34.42942,-118.64478,311.10,0.40,0.44,2024-09-01,09:20:00,N21,VIIRS,n,...,24.907000,30.207000,28.257000,25.357000,0.079,0.146,0.168,0.200,09:00:00,1
62049,34.60736,-117.33656,317.76,0.52,0.42,2024-09-01,09:20:00,N21,VIIRS,n,...,26.746000,32.146000,29.595999,26.146000,0.010,0.125,0.132,0.180,09:00:00,1


### Creating target label = 0

In [102]:
"""I am calculating how many instances in the label=1 dataset involve data from the years 2020, 2021, 2022, 2023, and 2024. 
I will use the same counts to sample from the weather data that does not have a match with NASA data (not related to wildfires).
This helps me create a balanced dataset with an equal number of wildfire and non-wildfire entries."""

'I am calculating how many instances in the label=1 dataset involve data from the years 2020, 2021, 2022, 2023, and 2024. \nI will use the same counts to sample from the weather data that does not have a match with NASA data (not related to wildfires).\nThis helps me create a balanced dataset with an equal number of wildfire and non-wildfire entries.'

In [103]:
positive_samples['acq_date']=pd.to_datetime(positive_samples['acq_date'])
positive_samples['year']=positive_samples['acq_date'].dt.year

In [104]:
# from 2020 to 2024
year_counts=positive_samples.groupby('year').size().tolist()
year_counts

[17023, 15169, 7106, 7270, 10380]

In [105]:
negative_samples=wildfire_df.merge(hourly_weather,left_on=['latitude','longitude','acq_date','nearest_hour'],right_on=['latitude','longitude','acq_date','time'],how='right').drop_duplicates()
negative_samples=negative_samples[negative_samples['bright_ti4'].isna()].drop(columns=wildfire_columns,axis=1).drop('nearest_hour',axis=1).drop_duplicates()
negative_samples['date']=pd.to_datetime(negative_samples['date'])
negative_samples['year']=negative_samples['date'].dt.year

In [106]:
start_year=2020
sampled_negatives=pd.DataFrame()
for val in year_counts:
    year_data=negative_samples[negative_samples['year']==start_year]
    # Randomly sample the specified count, or take all if not enough rows are available
    sampling_df = year_data.sample(n=min(val, len(year_data)), random_state=1)
    sampled_negatives=pd.concat([sampled_negatives,sampling_df])
    start_year+=1

In [107]:
sampled_negatives['fire']=0

In [108]:
training_df=pd.concat([positive_samples,sampled_negatives]).drop(columns=wildfire_df,axis=1).drop('year',axis=1)

In [109]:
training_df

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,time,fire
0,2020-01-02 09:00:00+00:00,10.460000,95.428820,9.7600,9.326071,0.0,0.0,0.0,0.0,0.0,...,10.760000,13.110001,14.010000,21.910000,0.275,0.318,0.195,0.026,09:00:00,1
1,2020-01-02 09:00:00+00:00,10.453500,95.428590,9.7535,9.317741,0.0,0.0,0.0,0.0,0.0,...,10.753500,13.103500,14.003500,21.903500,0.275,0.318,0.195,0.026,09:00:00,1
2,2020-01-02 09:00:00+00:00,10.414500,95.427210,9.7145,9.267784,0.0,0.0,0.0,0.0,0.0,...,10.714500,13.064501,13.964500,21.864500,0.275,0.318,0.195,0.026,09:00:00,1
3,2020-01-02 09:00:00+00:00,9.223500,99.663920,9.1735,8.143780,0.0,0.0,0.0,0.0,0.0,...,10.423500,12.823500,13.723500,21.923500,0.273,0.324,0.201,0.020,09:00:00,1
4,2020-01-02 09:00:00+00:00,9.243000,99.663990,9.1930,8.168585,0.0,0.0,0.0,0.0,0.0,...,10.443000,12.842999,13.743000,21.942999,0.273,0.324,0.201,0.020,09:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333864,2024-07-18 23:00:00+00:00,28.476500,26.921050,7.6265,26.616827,0.0,0.0,0.0,NaN,1.0,...,41.726498,30.776499,26.726500,18.426498,0.042,0.134,0.192,0.235,23:00:00,0
1282940,2024-07-09 03:00:00+00:00,25.435000,36.749325,9.5850,24.854360,0.0,0.0,0.0,0.0,0.0,...,32.185000,28.734999,23.435000,17.535000,0.135,0.170,0.229,0.293,03:00:00,0
1215274,2024-04-12 17:00:00+00:00,13.914001,82.397710,10.9640,13.423975,0.0,0.0,0.0,0.0,3.0,...,14.164001,13.364000,12.314000,12.314000,0.231,0.314,0.330,0.342,17:00:00,0
1410192,2024-07-29 07:00:00+00:00,7.604000,52.525845,-1.4960,4.411881,0.0,0.0,0.0,NaN,0.0,...,10.654000,14.604001,13.604000,7.954000,0.270,0.296,0.347,0.392,07:00:00,0


In [110]:
training_df.to_csv('../Datasets/training_df_raw.csv',index=False)